<a href="https://colab.research.google.com/github/francescopatane96/Bioactivity-prediction-with-ML/blob/main/ML_7_1_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download all public molecules in ZINC database (42271452 molecules), including around 1M compounds not in subset #6 that may not be available commercially (last update: 2014-11-24)

In [ ]:
!wget http://zinc12.docking.org/db/bysubset/10/10_p0.smi.gz    #download all public molecules in ZINC, including around 1M compounds not in subset #6 that may not be available commercially

In [129]:
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect

In [128]:
!wget http://zinc12.docking.org/db/bysubset/17/17_p0.smi.gz   #or download a fraction of it

--2022-09-04 15:04:46--  http://zinc12.docking.org/db/bysubset/17/17_p0.smi.gz
Resolving zinc12.docking.org (zinc12.docking.org)... 169.230.26.169
Connecting to zinc12.docking.org (zinc12.docking.org)|169.230.26.169|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2407726 (2.3M) [application/x-gzip]
Saving to: ‘17_p0.smi.gz’

17_p0.smi.gz        100%[===================>]   2.30M  4.46MB/s    in 0.5s    

2022-09-04 15:04:47 (4.46 MB/s) - ‘17_p0.smi.gz’ saved [2407726/2407726]



In [ ]:
!gunzip 10_p0.smi.gz  #obtain .smi file


gzip: 10_p0.smi.gz: No such file or directory


In [130]:
!gunzip 17_p0.smi.gz #''

In [131]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [132]:
import pickle

In [133]:
objects = []                         #read pickle object file (pkl). It is our model
with (open("IDH_model.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [134]:
%matplotlib inline

import os

from rdkit import rdBase

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from IPython.display import display

from rdkit.Chem import PandasTools
import pandas as pd
PandasTools.pd = pd

import numpy as np

np.random.seed(69)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

#from bokeh import mpl
#from bokeh.plotting import output_notebook, show

print("RDKit Version: %s" % rdBase.rdkitVersion)

RDKit Version: 2022.03.5


In [135]:
smiles_file = os.path.join(os.path.abspath(os.curdir) ,"17_p0.smi")

mols_dict = dict()
bad_mols = 0
bad_smiles = list()
dup_mols = 0
with open(smiles_file, "r") as fd:
    header = fd.readline()
    for line in fd.readlines():
        line = line.split()
        #print line
        # Add molecule information in a dictionary...
        # SMILES: (Mol_Id, Mol_Obj)
        smiles = line[0]
        mol_id = line[1]
        mol_obj = Chem.MolFromSmiles(smiles)
        # Skip bad molecules...
        if mol_obj is None:
            bad_mols += 1
            bad_smiles.append(smiles)
            continue
        if smiles in mols_dict:
            dup_mols += 1
            continue
        mols_dict[smiles] = (mol_id, mol_obj)
    # end for
# end with

print("Bad Mols: %s" % bad_mols)
print("Bad SMILES: %s" % bad_smiles)
print("Duplicate Mols: %s" % dup_mols)
print(len(mols_dict))

Bad Mols: 5
Bad SMILES: ['CC1=CC=N(=C(C#N)C#N)C=C1', 'c1nc(nn1CCC#N)N=N=N', 'c1ccc(cc1)C=N2=CC=CC=C2', 'C1=CC=N(=C(C#N)C#N)C=C1', 'c1cc(cc(c1)Cl(=O)(=O)=O)N']
Duplicate Mols: 0
325944


In [136]:
import random

In [137]:

selected_mols = random.choices(list(mols_dict.keys()), k=100)           #select 100 molecules of the dictionary, convert to list to output
selected_mols

['C[C@H]([C@@](C)(C[NH3+])c1ccccc1)O',
 'C[C@H]1CCC[NH+]1[C@@H]2CC[NH2+]C2',
 'Cc1cc2c([nH]1)C[NH2+][C@@H](C2)C',
 'Cc1ncc(s1)C(=O)[O-]',
 'CCN(CC)c1ccc(nc1)Cl',
 'CC1(CCC(=O)C[C@H]1C#N)C',
 'C1CC2(CCC[NH+]2C1)C(=O)[O-]',
 'C[C@H]1COc2cscc2O1',
 'c1ccc(cc1)[C@@H](CO)[NH2+]C2CC2',
 'CC1=[NH+][C@@H](CCN1C)C(=O)N',
 'Cc1cc(c(o1)C)[C@H](CC2CC2)[NH3+]',
 'c1c(nc([nH]c1=O)N)N',
 'c1cncc2c1C[C@@H](S2)C#N',
 'CC[C@@H](c1ccncn1)[C@@H](C)[NH3+]',
 'Cn1cc(cn1)Cn2cc(cn2)N',
 'Cc1c(c(cc([nH+]1)C(=O)OC)N)N',
 'c1c(nc(o1)C2CCC2)CC(=O)[O-]',
 'C[C@H](Cc1nc(on1)COC)[NH3+]',
 'CCc1nccc(n1)C[C@@H](CC)[NH3+]',
 'c1ccc(cc1)C2C(=O)CCC2=O',
 'c1c[nH]c2c1cncc2C#N',
 'CCC1CCC(CC1)[C@@H](C(=O)[O-])[NH3+]',
 'C[C@H]([C@@H]1CCCO1)NC(=O)C2CC2',
 'c1cc(cc(c1)OCCO)CO',
 'c1c2c(n[nH]1)CC[C@@H]2[NH3+]',
 'C[NH2+][C@@H]([C@H]1CCOC1)C(=O)N',
 'C[C@H](CC[NH2+]C(C)(C)C)[NH3+]',
 'C[C@H]1C(=[NH2+])N(C(=[NH+]1)CC(=O)OC)C',
 'Cc1c(c[nH]n1)[C@@H]2C[C@@H](CC[NH2+]2)O',
 'Cc1ccnc2c1[C@@H](CNC2)O',
 'c1cc(=O)ccc2c1cco2',
 'C1C[C

In [138]:
def listToDict(lst):
    op = dict.fromkeys(lst, 5)
    return op

In [139]:
selected_mols = listToDict(selected_mols)             #print the casual dictionary of 100 molecules
selected_mols


{'C[C@H]([C@@](C)(C[NH3+])c1ccccc1)O': 5,
 'C[C@H]1CCC[NH+]1[C@@H]2CC[NH2+]C2': 5,
 'Cc1cc2c([nH]1)C[NH2+][C@@H](C2)C': 5,
 'Cc1ncc(s1)C(=O)[O-]': 5,
 'CCN(CC)c1ccc(nc1)Cl': 5,
 'CC1(CCC(=O)C[C@H]1C#N)C': 5,
 'C1CC2(CCC[NH+]2C1)C(=O)[O-]': 5,
 'C[C@H]1COc2cscc2O1': 5,
 'c1ccc(cc1)[C@@H](CO)[NH2+]C2CC2': 5,
 'CC1=[NH+][C@@H](CCN1C)C(=O)N': 5,
 'Cc1cc(c(o1)C)[C@H](CC2CC2)[NH3+]': 5,
 'c1c(nc([nH]c1=O)N)N': 5,
 'c1cncc2c1C[C@@H](S2)C#N': 5,
 'CC[C@@H](c1ccncn1)[C@@H](C)[NH3+]': 5,
 'Cn1cc(cn1)Cn2cc(cn2)N': 5,
 'Cc1c(c(cc([nH+]1)C(=O)OC)N)N': 5,
 'c1c(nc(o1)C2CCC2)CC(=O)[O-]': 5,
 'C[C@H](Cc1nc(on1)COC)[NH3+]': 5,
 'CCc1nccc(n1)C[C@@H](CC)[NH3+]': 5,
 'c1ccc(cc1)C2C(=O)CCC2=O': 5,
 'c1c[nH]c2c1cncc2C#N': 5,
 'CCC1CCC(CC1)[C@@H](C(=O)[O-])[NH3+]': 5,
 'C[C@H]([C@@H]1CCCO1)NC(=O)C2CC2': 5,
 'c1cc(cc(c1)OCCO)CO': 5,
 'c1c2c(n[nH]1)CC[C@@H]2[NH3+]': 5,
 'C[NH2+][C@@H]([C@H]1CCOC1)C(=O)N': 5,
 'C[C@H](CC[NH2+]C(C)(C)C)[NH3+]': 5,
 'C[C@H]1C(=[NH2+])N(C(=[NH+]1)CC(=O)OC)C': 5,
 'Cc1c(c[nH]n1)[C@

In [140]:

df = pd.Series(selected_mols).rename_axis('smiles').reset_index(name='keys')
df

,smiles,keys
0,C[C@H]([C@@](C)(C[NH3+])c1ccccc1)O,5
1,C[C@H]1CCC[NH+]1[C@@H]2CC[NH2+]C2,5
2,Cc1cc2c([nH]1)C[NH2+][C@@H](C2)C,5
3,Cc1ncc(s1)C(=O)[O-],5
4,CCN(CC)c1ccc(nc1)Cl,5
...,...,...
95,Cc1cnnc(n1)[C@@H]([C@H](C)[NH3+])O,5
96,c1cnn(c1)c2ccnc(c2)C[NH3+],5
97,CC(C)C[C@H](CO)[NH3+],5
98,c1cc(cc(c1)Cl)C(C#N)(F)F,5


In [141]:
df2 = df.drop(columns=['keys'])
df2


,smiles
0,C[C@H]([C@@](C)(C[NH3+])c1ccccc1)O
1,C[C@H]1CCC[NH+]1[C@@H]2CC[NH2+]C2
2,Cc1cc2c([nH]1)C[NH2+][C@@H](C2)C
3,Cc1ncc(s1)C(=O)[O-]
4,CCN(CC)c1ccc(nc1)Cl
...,...
95,Cc1cnnc(n1)[C@@H]([C@H](C)[NH3+])O
96,c1cnn(c1)c2ccnc(c2)C[NH3+]
97,CC(C)C[C@H](CO)[NH3+]
98,c1cc(cc(c1)Cl)C(C#N)(F)F


In [142]:
def smiles_to_fp(smiles, method="maccs", n_bits=2048):
    """
    Encode a molecule from a SMILES string into a fingerprint.

    Parameters
    ----------
    smiles : str
        The SMILES string defining the molecule.

    method : str
        The type of fingerprint to use. Default is MACCS keys.

    n_bits : int
        The length of the fingerprint.

    Returns
    -------
    array
        The fingerprint array.

    """

    # convert smiles to RDKit mol object
    mol = Chem.MolFromSmiles(smiles)

    if method == "maccs":
        return np.array(MACCSkeys.GenMACCSKeys(mol))
    if method == "morgan2":
        return np.array(GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits))
    if method == "morgan3":
        return np.array(GetMorganFingerprintAsBitVect(mol, 3, nBits=n_bits))
    else:
        # NBVAL_CHECK_OUTPUT
        print(f"Warning: Wrong method specified: {method}. Default will be used instead.")
        return np.array(MACCSkeys.GenMACCSKeys(mol))

In [143]:
# Add column for fingerprint
df["fp"] = df["smiles"].apply(smiles_to_fp)
df.head(3)

,smiles,keys,fp
0,C[C@H]([C@@](C)(C[NH3+])c1ccccc1)O,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,C[C@H]1CCC[NH+]1[C@@H]2CC[NH2+]C2,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Cc1cc2c([nH]1)C[NH2+][C@@H](C2)C,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [177]:
df2 = df.drop(columns=['keys', 'smiles'])
df2

,fp
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
95,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
96,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
97,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
98,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [156]:
df2[0:1]

,fp
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [178]:
X = df2.to_csv('drugs.csv')



In [195]:
x = pd.read_csv('drugs.csv')
x

,Unnamed: 0,fp
0,0,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
1,1,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2,2,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,3,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
4,4,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
...,...,...
95,95,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
96,96,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
97,97,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
98,98,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...


In [210]:
x1 = x.replace(to_replace='\[', value="", regex=True)
x1

,Unnamed: 0,fp
0,0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,2,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,3,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,4,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...
95,95,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
96,96,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
97,97,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
98,98,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [212]:
x2 = x1.replace(to_replace='\]', value="", regex=True)
x2

,Unnamed: 0,fp
0,0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,2,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,3,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,4,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
...,...,...
95,95,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
96,96,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
97,97,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
98,98,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [ ]:
x2 = x1.replace(to_replace='\]', value="", regex=True)
x2

In [215]:
c = x2.to_csv('drugs.csv')

In [228]:
x3 = x2.to_numpy()
x3



array([[0,
        '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0\n 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0\n 0 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0'],
       [1,
        '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0\n 0 1 0 0 0 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0\n 1 0 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1\n 1 0 1 1 0 1 0 0 1 0 1 0 1 1 0 0 0 1 0'],
       [2,
        '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0\n 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0\n 1 0 0 0 1 1 0 0 0 1 1 

In [203]:
with open('IDH_model.pkl', 'rb') as f:
    loaded_classifier = pickle.load(f)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [226]:

objects = pd.read_pickle("IDH_model.pkl")
objects

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [229]:
x = pd.read_csv('foglio.csv')

entry_pred = loaded_classifier.predict([x3])
print("pIC50 prediction", entry_pred)

ValueError: ignored

In [160]:
y = loaded_classifier.predict(df3[0:1])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


ValueError: ignored

In [ ]:
def build_model(input_data):
    # Reads in saved regression model
    load_model = pickle.load(open('IDH_model.pkl', 'rb'))
    # Apply model to make predictions
    prediction = load_model.predict(input_data)
    st.header('**Prediction output**')
    prediction_output = pd.Series(prediction, name='pIC50')
    molecule_name = pd.Series(load_data[1], name='molecule_name')
    df = pd.concat([molecule_name, prediction_output], axis=1)
    st.write(df)
    st.markdown(filedownload(df), unsafe_allow_html=True)

In [ ]:
load_data = pd.read_table('file.smi', sep=' ', header=None)
load_data.to_csv('molecule.smi', sep = '\t', header = False, index = False)

In [ ]:
df3.to_csv('drugs.csv')

In [ ]:

Xlist = list(pd.read_csv('drugs.csv').columns)
Xlist



['Unnamed: 0', 'fp']

In [ ]:
build_model(Xlist)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


ValueError: ignored